In [ ]:
import pandas as pd
import numpy as np
reviews = pd.read_csv('/content/tripadvisor_hotel_reviews - tripadvisor_hotel_reviews.csv')
reviews.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
reviews['Label'] = reviews.Rating >3
reviews.head()


,Review,Rating,Label
0,nice hotel expensive parking got good deal sta...,4,True
1,ok nothing special charge diamond member hilto...,2,False
2,nice rooms not 4* experience hotel monaco seat...,3,False
3,"unique, great stay, wonderful time hotel monac...",5,True
4,"great stay great stay, went seahawk game aweso...",5,True


In [ ]:
# reviews['Label'] = reviews['Label'].astype(int)
reviews = reviews.replace({True: 'pos', False: 'neg'})

In [ ]:
reviews.dtypes
reviews.head()
reviews.isna().sum()
len(reviews)
reviews['Rating'].describe()

count    20491.000000
mean         3.952223
std          1.233030
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: Rating, dtype: float64

In [ ]:
blanks = []  # start with an empty list

for i,rv,rt,lb in reviews.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

print(len(blanks), 'blanks: ', blanks)
reviews['Label'].value_counts()

0 blanks:  []


pos    15093
neg     5398
Name: Label, dtype: int64

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sid.polarity_scores(reviews.loc[0]['Review'])
reviews.loc[0]['Label']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


'pos'

In [ ]:
reviews['Scores'] = reviews['Review'].apply(lambda Review: sid.polarity_scores(Review))
reviews.head()


,Review,Rating,Label,Scores
0,nice hotel expensive parking got good deal sta...,4,pos,"{'neg': 0.072, 'neu': 0.643, 'pos': 0.285, 'co..."
1,ok nothing special charge diamond member hilto...,2,neg,"{'neg': 0.11, 'neu': 0.701, 'pos': 0.189, 'com..."
2,nice rooms not 4* experience hotel monaco seat...,3,neg,"{'neg': 0.081, 'neu': 0.7, 'pos': 0.219, 'comp..."
3,"unique, great stay, wonderful time hotel monac...",5,pos,"{'neg': 0.06, 'neu': 0.555, 'pos': 0.385, 'com..."
4,"great stay great stay, went seahawk game aweso...",5,pos,"{'neg': 0.135, 'neu': 0.643, 'pos': 0.221, 'co..."


In [ ]:
reviews['compound']  = reviews['Scores'].apply(lambda score_dict: score_dict['compound'])

reviews.head()

,Review,Rating,Label,Scores,compound
0,nice hotel expensive parking got good deal sta...,4,pos,"{'neg': 0.072, 'neu': 0.643, 'pos': 0.285, 'co...",0.9747
1,ok nothing special charge diamond member hilto...,2,neg,"{'neg': 0.11, 'neu': 0.701, 'pos': 0.189, 'com...",0.9787
2,nice rooms not 4* experience hotel monaco seat...,3,neg,"{'neg': 0.081, 'neu': 0.7, 'pos': 0.219, 'comp...",0.9889
3,"unique, great stay, wonderful time hotel monac...",5,pos,"{'neg': 0.06, 'neu': 0.555, 'pos': 0.385, 'com...",0.9912
4,"great stay great stay, went seahawk game aweso...",5,pos,"{'neg': 0.135, 'neu': 0.643, 'pos': 0.221, 'co...",0.9797


In [ ]:
reviews['comp_score'] = reviews['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

reviews.head()

,Review,Rating,Label,Scores,compound,comp_score
0,nice hotel expensive parking got good deal sta...,4,pos,"{'neg': 0.072, 'neu': 0.643, 'pos': 0.285, 'co...",0.9747,pos
1,ok nothing special charge diamond member hilto...,2,neg,"{'neg': 0.11, 'neu': 0.701, 'pos': 0.189, 'com...",0.9787,pos
2,nice rooms not 4* experience hotel monaco seat...,3,neg,"{'neg': 0.081, 'neu': 0.7, 'pos': 0.219, 'comp...",0.9889,pos
3,"unique, great stay, wonderful time hotel monac...",5,pos,"{'neg': 0.06, 'neu': 0.555, 'pos': 0.385, 'com...",0.9912,pos
4,"great stay great stay, went seahawk game aweso...",5,pos,"{'neg': 0.135, 'neu': 0.643, 'pos': 0.221, 'co...",0.9797,pos


In [ ]:
reviews.comp_score.value_counts()

pos    18874
neg     1617
Name: comp_score, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(reviews['Label'], reviews['comp_score'])

0.805133961251281

In [ ]:
print(classification_report(reviews['Label'],reviews['comp_score']))
print(confusion_matrix(reviews['Label'],reviews['comp_score']))

              precision    recall  f1-score   support

         neg       0.93      0.28      0.43      5398
         pos       0.79      0.99      0.88     15093

    accuracy                           0.81     20491
   macro avg       0.86      0.64      0.66     20491
weighted avg       0.83      0.81      0.76     20491

[[ 1511  3887]
 [  106 14987]]
